In [3]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")


Using TensorFlow backend.


In [6]:
# Configurations
class BasketballConfig(Config):
    NAME = "basketball"
    
    # NUM_CLASSES = ?
    # STEPS_PER_EPOCH


In [21]:
f = open('allImages.json')
fla = f.read()
mydict = json.loads(fla)

dict_keys(['frame16848.jpg.json', 'frame19608.jpg.json', 'frame1032.jpg.json', 'frame11988.jpg.json', 'frame9144.jpg.json', 'frame192.jpg.json', 'frame14376.jpg.json', 'frame13584.jpg.json', 'frame4308.jpg.json', 'frame15024.jpg.json', 'frame10284.jpg.json', 'frame11880.jpg.json', 'frame11676.jpg.json', 'frame18360.jpg.json', 'frame10524.jpg.json', 'frame15972.jpg.json', 'frame4200.jpg.json', 'frame13224.jpg.json', 'frame7500.jpg.json', 'frame19032.jpg.json', 'frame1608.jpg.json', 'frame11004.jpg.json', 'frame7440.jpg.json', 'frame9096.jpg.json', 'frame10464.jpg.json', 'frame10356.jpg.json', 'frame1824.jpg.json', 'frame12036.jpg.json', 'frame11736.jpg.json', 'frame984.jpg.json', 'frame16800.jpg.json', 'frame5628.jpg.json', 'frame7548.jpg.json', 'frame6780.jpg.json', 'frame11952.jpg.json', 'frame4248.jpg.json', 'frame16908.jpg.json', 'frame12960.jpg.json', 'frame19104.jpg.json', 'frame9648.jpg.json', 'frame12984.jpg.json', 'frame18564.jpg.json', 'frame15180.jpg.json', 'frame10320.jpg.js

In [23]:
# Dataset

class BasketballDataset(utils.Dataset):
    def load_basketball(self, dataset_dir, subset):
        # load a subset of the basketball dataset
        # dataset_dir: Root directory of dataset, subset: subset to load, either training or validation
        
        # Add classes
        self.add_class("basketball_player", 1, "basketball_player")
        
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        
        # Load annotations. Each image has four attributes: tags, description, objects, and size. 
       
        annotations = json.load(open(os.path.join(dataset_dir, "allImages.json")))
        annotations = list(annotations.values())  # don't need the dict keys


        # Add images
        for a in annotations:
            polygons =[]
            if type[a['description']] is dict:
                polygons_diagonal = [r['exterior'] for r in a['description'].values()]
            else:
                polygons_diagonal = [r['exterior'] for r in a['description'] 
            for (x,y) in polygons_diagonal:
                listx = []
                listy = []
                listx.add(x)
                listy.add(y)
                for i in range(len(listx)-1):
                    polygons.add(listx[i],listy[i])
                    polygons.add(listx[i+1],listy[i])
                    polygons.add(listx[i+1], listy[i+1])
                    polygons.add(listx[i], listy[i+1])
                listx.clear()
                listy.clear()
            
                    
    
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
                                     
            # need to set key = img files
                image_path = os.path.join(dataset_dir, key)
                image = skimage.io.imread(image_path)
                height = a['size'][0]
                width = a['size'][1]

            self.add_image(
                "basketball_player",
                image_id= key,  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a basketball dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "basketball":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "basketball":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


    def train(model):
    """Train the model."""
    # Training dataset.
        dataset_train = BasketballDataset()
        dataset_train.load_basketball(args.dataset, "Train")
        dataset_train.prepare()

    # Validation dataset
        dataset_val = BasketballDataset()
        dataset_val.load_basketball(args.dataset, "Val")
        dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
        print("Training network heads")
        model.train(dataset_train, dataset_val,learning_rate=config.LEARNING_RATE,epochs=30,layers='heads')


    def color_splash(image, mask):
    """Apply color splash effect.
    image: RGB image [height, width, 3]
    mask: instance segmentation mask [height, width, instance count]

    Returns result image.
    """
    # Make a grayscale copy of the image. The grayscale copy still
    # has 3 RGB channels, though.
        gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image)) * 255
    # Copy color pixels from the original color image where mask is set
        if mask.shape[-1] > 0:
        # We're treating all instances as one, so collapse the mask into one layer
            mask = (np.sum(mask, -1, keepdims=True) >= 1)
            splash = np.where(mask, image, gray).astype(np.uint8)
        else:
            splash = gray.astype(np.uint8)
        return splash


    def detect_and_color_splash(model, image_path=None, video_path=None):
        assert image_path or video_path

    # Image or video?
        if image_path:
        # Run model detection and generate the color splash effect
            print("Running on {}".format(args.image))
        # Read image
            image = skimage.io.imread(args.image)
        # Detect objects
            r = model.detect([image], verbose=1)[0]
        # Color splash
            splash = color_splash(image, r['masks'])
        # Save output
            file_name = "splash_{:%Y%m%dT%H%M%S}.png".format(datetime.datetime.now())
            skimage.io.imsave(file_name, splash)
        elif video_path:
            import cv2
        # Video capture
            vcapture = cv2.VideoCapture(video_path)
            width = int(vcapture.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(vcapture.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = vcapture.get(cv2.CAP_PROP_FPS)

        # Define codec and create video writer
            file_name = "splash_{:%Y%m%dT%H%M%S}.avi".format(datetime.datetime.now())
            vwriter = cv2.VideoWriter(file_name,cv2.VideoWriter_fourcc(*'MJPG'),fps, (width, height))

            count = 0
            success = True
            while success:
                print("frame: ", count)
            # Read next image
                success, image = vcapture.read()
                if success:
                # OpenCV returns images as BGR, convert to RGB
                    image = image[..., ::-1]
                # Detect objects
                    r = model.detect([image], verbose=0)[0]
                # Color splash
                    splash = color_splash(image, r['masks'])
                # RGB -> BGR to save image to video
                    splash = splash[..., ::-1]
                # Add image to video writer
                    vwriter.write(splash)
                    count += 1
            vwriter.release()
        print("Saved to ", file_name)


############################################################
    #  Training
############################################################

    if __name__ == '__main__':
        import argparse

    # Parse command line arguments
        parser = argparse.ArgumentParser(
            description='Train Mask R-CNN to detect basketball players and the ball.')
        parser.add_argument("command",metavar="<command>",
                        help="'train' or 'splash'")
        parser.add_argument('--dataset', required=False,
                        metavar="/path/to/basketball/dataset/",
                        help='Directory of the Basketball dataset')
        parser.add_argument('--weights', required=True,
                        metavar="/path/to/weights.h5",
                        help="Path to weights .h5 file or 'coco'")
        parser.add_argument('--logs', required=False,
                        default=DEFAULT_LOGS_DIR,
                        metavar="/path/to/logs/",
                        help='Logs and checkpoints directory (default=logs/)')
        parser.add_argument('--image', required=False,
                        metavar="path or URL to image",
                        help='Image to apply the color splash effect on')
        parser.add_argument('--video', required=False,
                        metavar="path or URL to video",
                        help='Video to apply the color splash effect on')
        args = parser.parse_args()

    # Validate arguments
        if args.command == "train":
            assert args.dataset, "Argument --dataset is required for training"
        elif args.command == "splash":
            assert args.image or args.video,\
                   "Provide --image or --video to apply color splash"

        print("Weights: ", args.weights)
        print("Dataset: ", args.dataset)
        print("Logs: ", args.logs)

    # Configurations
        if args.command == "train":
            config = BasketballConfig()
        else:
            class InferenceConfig(BasketballConfig):
            # Set batch size to 1 since we'll be running inference on
            # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
                GPU_COUNT = 1
                IMAGES_PER_GPU = 1
            config = InferenceConfig()
        config.display()

    # Create model
        if args.command == "train":
            model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=args.logs)
        else:
            model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir=args.logs)

    # Select weights file to load
        if args.weights.lower() == "coco":
            weights_path = COCO_WEIGHTS_PATH
        # Download weights file
            if not os.path.exists(weights_path):
                utils.download_trained_weights(weights_path)
        elif args.weights.lower() == "last":
        # Find last trained weights
            weights_path = model.find_last()
        elif args.weights.lower() == "imagenet":
        # Start from ImageNet trained weights
            weights_path = model.get_imagenet_weights()
        else:
            weights_path = args.weights

    # Load weights
        print("Loading weights ", weights_path)
        if args.weights.lower() == "coco":
        # Exclude the last layers because they require a matching
        # number of classes
            model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
        else:
            model.load_weights(weights_path, by_name=True)

    # Train or evaluate
        if args.command == "train":
            train(model)
        elif args.command == "splash":
            detect_and_color_splash(model, image_path=args.image,
                                video_path=args.video)
        else:
            print("'{}' is not recognized. "
                  "Use 'train' or 'splash'".format(args.command))
